# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
# from config import gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_to_load = "CityWeather.csv"
weather_data = pd.read_csv(weather_data_to_load)
weather_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Rikitea,21,PF,1590384545,72,-23.12,-134.97,72.09,25.26
1,Tuktoyaktuk,20,CA,1590384545,74,69.45,-133.04,37.40,14.99
2,Coro,70,VE,1590384545,81,11.40,-69.67,77.92,15.05
3,Xapuri,43,BR,1590384545,75,-10.65,-68.50,60.30,2.42
4,Busselton,95,AU,1590384292,83,-33.65,115.33,53.01,29.46


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"].astype(float)

# # Plot Heatmap
fig = gmaps.figure()

# # Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
blue_skies_cities = weather_data[weather_data['Cloudiness'] == 0]
crisp_cities = weather_data[weather_data['Humidity'].between(0, 30)]
cool_cities = weather_data[weather_data['Max Temp'].between(65, 77)]
perfect_hairdo_cities = weather_data[weather_data['Wind Speed'].between(0, 10)]

vacation_cities = pd.DataFrame ({
    'City': ['City'], 
    'Cloudiness': [blue_skies_cities], 
    'Country': ['Country'],
    'Date': ['Date'], 
    'Humidity': [crisp_cities], 
    'Lat': ['Lat'], 
    'Lng': ['Lng'], 
    'Max Temp': [cool_cities], 
    'Wind Speed': [perfect_hairdo_cities]
})




# df = pd.DataFrame(d1)
# print(df)
# # drop all rows with any NaN and NaT values
# df1 = df.dropna()
# print(df1)



# City	Cloudiness	Country	Date	Humidity	Lat	Lng	Max Temp	Wind Speed

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
